In [90]:
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np

In [6]:
api_dashboard = 'V6lMawCCbCP2rShqtU3TmYCY3Em1Osd8I2DLYxwr'

## Functions

In [49]:
def load_factchecks():
    factchecks = pd.read_excel('../../../data/1-factchecks/2-clean_factchecks/subset_factchecks_data.xlsx')
    factchecks = factchecks.drop(['Unnamed: 0'], axis = 1)
    return(factchecks)

def obtain_engagement(link, id_factcheck):
    
    api_dashboard = 'V6lMawCCbCP2rShqtU3TmYCY3Em1Osd8I2DLYxwr'
    
    URL_BASE = "https://api.crowdtangle.com/links"
    PARAMS = {'link': link, 'count': 1000,'token': api_dashboard, 'platforms': 'facebook'}
    
    r = requests.get(url = URL_BASE, params=PARAMS)
    data = r.json()
    try:
        df = pd.DataFrame(data['result']['posts'])
        df['id_factcheck'] = id_factcheck
        return(df)
    except:
        pass

In [67]:
def obtain_stats(all_engaements):
    users = all_engaements[['id', 'id_factcheck']].groupby(['id_factcheck']).count().reset_index().rename(columns = {'id':'number_users'})
    
    all_engaements['interactions'] = all_engaements['statistics'].apply(lambda x: sum(x['actual'].values()))
    interactions = all_engaements[['interactions', 'id_factcheck']].groupby(['id_factcheck']).sum().reset_index()
    
    stats_df = pd.merge(left= users, 
            right = interactions, 
            on = 'id_factcheck', 
            how = 'outer')
    
    stats_df['total_engagement'] = stats_df.apply(lambda x: x['number_users'] + 
                                                  x['interactions'], axis = 1)
    return(stats_df)
    
    

## Apply Functions

In [23]:
factchecks = load_factchecks()

In [50]:
df_engagements = []
for x, y in tqdm(zip(factchecks.link_chequeo, factchecks.id_factcheck), total = len(factchecks)):
    df_engagements.append(obtain_engagement(x, y))


100%|██████████| 562/562 [17:06<00:00,  1.83s/it]


In [51]:
all_engaements = pd.concat(df_engagements)

In [68]:
stat_engagements = obtain_stats(all_engaements)

In [100]:
complete_data = pd.merge(left = factchecks, 
        right = stat_engagements, 
        on = 'id_factcheck', 
        how = 'left')


In [101]:
complete_data['total_engagement'] = complete_data['total_engagement'].fillna(0)

In [102]:
complete_data['above_median'] = np.where(complete_data['total_engagement'] > complete_data.total_engagement.median(), 1, 0)

In [104]:
complete_data.to_excel('../../../data/1-factchecks/2-clean_factchecks/factchecks_engagements.xlsx')